**PALS LLM API Framework**

Question: Can we use GenAI to perform risk assessments given data to accurately decide whether a person is at risk for hospitalization for a mood disorder? <br>
    - What would the implications be for equitable health care if AI methods were implemented in Healthcare? <br>
    - What does it mean to have a 'successful' mental health chatbot? What if parameters are tweaked? What parameters would I need to look into? <br>

In [11]:
%pip install openai


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY=os.getenv("OPEN_ROUTER_API_KEY")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)


In [22]:
#first attempt -- topic modeling

def patient_risk_simulation(model, prompt):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": (
                    "Assign a numerical risk score from 0–10 (10 being the highest risk) " #Is 0-10 the best range to work with?
                    "for hospitalization for a person with the following characteristics: "
                    f"{prompt}. "
                    "Respond only with:\n"
                    "1. Risk Assessment Score\n"
                    "2. A list of parameters and their associated numerical positive or negative risk values "
                    "that add up to the Risk Assessment Score\n"
                    "3. A one to two sentence rationale.\n"
                    "Exclude baseline risk in all calculations."
                )
            }
        ]
    )
    
    return completion.choices[0].message.content


In [23]:
print(patient_risk_simulation("openai/gpt-oss-20b:free", "Female, Age=42, Weight=105lbs, Type 2 Diabetes, 50mg of Prozac"))

**Risk Assessment Score:** 3  

**Parameters & Risk Values:**  
- Type 2 Diabetes: +3  
- Prozac (50 mg): +1  
- Low body weight (105 lb): –1  

**Rationale:**  
Type 2 diabetes is a significant contributor to hospitalization risk, while a moderate dose of Prozac adds a small additional risk; the low body weight slightly mitigates overall risk, yielding a moderate score of 3.


Now that the model is set up to perform hospital risk scores, I want to explore what typically goes into mood order diagnosis. This would require some external research to ensure that I use the right metrics in psychiatry to determine whether or not a patient would be considered at risk before devising a treatment plan.

I want to also look at other methods in prompt engineering --> chaining prompts, parameter efficient fine tuning (PEFT), etc
I also want to look at whether for certain demographic factors if the models reasoning only encompases one or two topics.

*Here's another testable /important research question: are there demographics where the reasoning used is less varied (e.g. for demographic X it considers 10 factors in hospitalization risk, but for demographic Y, no matter how much it's queried, the reasoning only encompasses 1 or 2 topics).


Question: Is there a presense of bias in medically applied LLM's? Can this be attributed to the training data?

Task: Create randomly generated synthetic data as a csv f including variables such as Age, BMI, health history (prior health issues), current diagnosis (if any), Race, Gender, Sexual Orientation, Weekly Alcohol Intake, Smoking, Political Ideology, Been feeling sad or fatugued in past two weeks, loss of interest in the past two weeks, hours of sleep, employment, socioeconomic status.

In [19]:
import pandas as pd
import numpy as np

np.random.seed(51)

# Define population size
n = 50  # you can scale this later

# Define synthetic distributions
ages = np.random.randint(18, 90, n)
bmis = np.round(np.random.normal(27, 5, n), 1)
health_history = np.random.choice(['Diabetes', 'Hypertension', 'None', 'Asthma', 'Heart Disease'], n)
current_diagnosis = np.random.choice(['None', 'Anxiety', 'Depression', 'Bipolar', 'Scizophrenia'], n)
race = np.random.choice(['White', 'Black', 'Asian', 'Hispanic', 'Native American', 'Other'], n)
gender = np.random.choice(['Male', 'Female', 'Non-binary', 'Other'], n)
sexual_orientation = np.random.choice(['Heterosexual', 'Homosexual', 'Bisexual', 'Asexual', 'Other'], n)
weekly_alcohol_intake = np.random.poisson(3, n)
smoking = np.random.choice(['Yes', 'No', 'Former Smoker'], n)
political_ideology = np.random.choice(['Liberal', 'Moderate', 'Conservative', 'Apolitical'], n)
sad_fatigued = np.random.choice(['Yes', 'No'], n)
loss_interest = np.random.choice(['Yes', 'No'], n)
hours_sleep = np.round(np.random.normal(7, 1.5, n), 1)
employment = np.random.choice(['Employed', 'Unemployed', 'Student', 'Retired'], n)
socioeconomic_status = np.random.choice(['Low', 'Middle', 'High'], n)

# Combine into a DataFrame
df = pd.DataFrame({
    'Age': ages,
    'BMI': bmis,
    'Health_History': health_history,
    'Current_Diagnosis': current_diagnosis,
    'Race': race,
    'Gender': gender,
    'Sexual_Orientation': sexual_orientation,
    'Weekly_Alcohol_Intake': weekly_alcohol_intake,
    'Smoking': smoking,
    'Political_Ideology': political_ideology,
    'Been_Sad_or_Fatigued': sad_fatigued,
    'Loss_of_Interest': loss_interest,
    'Hours_of_Sleep': hours_sleep,
    'Employment': employment,
    'Socioeconomic_Status': socioeconomic_status
})

# Save to CSV
df.to_csv('synthetic_data.csv', index=False)
df = pd.read_csv('synthetic_data.csv')
df.head()

,Age,BMI,Health_History,Current_Diagnosis,Race,Gender,Sexual_Orientation,Weekly_Alcohol_Intake,Smoking,Political_Ideology,Been_Sad_or_Fatigued,Loss_of_Interest,Hours_of_Sleep,Employment,Socioeconomic_Status
0,75,31.1,Heart Disease,NaN,Hispanic,Female,Asexual,2,No,Apolitical,No,No,7.1,Retired,Middle
1,87,21.3,Asthma,Scizophrenia,Black,Male,Asexual,0,Former Smoker,Conservative,Yes,Yes,9.8,Retired,Low
2,34,24.9,Asthma,NaN,Asian,Non-binary,Homosexual,2,No,Apolitical,Yes,No,5.7,Retired,Low
3,39,19.4,Diabetes,Bipolar,White,Female,Bisexual,2,No,Conservative,Yes,No,5.4,Unemployed,High
4,70,22.6,Asthma,Bipolar,Native American,Other,Bisexual,2,Yes,Moderate,Yes,No,8.4,Student,High


From the first five rows, prior to adding the conditional distributions, I noticed discreprenscies in which synthetic data might not be evaluative of a general populations. For instance, the political ideology and sexual orientation are factors that tend to relate, whereas in this case the randomization is almost too random to the point that the patients are almost unrealistic. I'm curious however to see what the LLM does with these patients that are not the typical average and how much each factor plays a role in its decision making process.

In [24]:
#Dependencies
import pandas as pd
import numpy as np

np.random.seed(51)

# population 
n = 50  

# Sample SES (socioecon status) first
ses_choices = ['Low', 'Middle', 'High']
ses_probs = [0.3, 0.5, 0.2]  # change to match your target population
ses = np.random.choice(ses_choices, size=n, p=ses_probs)

# Conditional sampling for Smoking given SES
# Specify P(smoker | SES)
p_smoke_given_ses = {
    'Low': 0.35,
    'Middle': 0.20,
    'High': 0.10,
}

smoking = []
for s in ses:
    smoking.append(np.random.choice(['Yes', 'No', 'Former'],
                                    p=[p_smoke_given_ses[s], 1 - p_smoke_given_ses[s], 0.0]))
smoking = np.array(smoking)

# Sample Sexual Orientation
sex_orient_choices = ['Heterosexual', 'Homosexual', 'Bisexual', 'Asexual', 'Other']
sex_orient_probs = [0.75, 0.07, 0.10, 0.03, 0.05]
sex_orient = np.random.choice(sex_orient_choices, size=n, p=sex_orient_probs)

# Enforce a 90% "alignment" between Sexual Orientation and Political Ideology.
# Define for each sexual orientation a dominant ideology with P=0.9, otherwise sample from remaining distribution.
dominant_ideology = {
    'Heterosexual': 'Conservative',
    'Homosexual': 'Liberal',
    'Bisexual': 'Liberal',
    'Asexual': 'Apolitical',
    'Other': 'Moderate'
}

other_ideology_choices = ['Liberal', 'Moderate', 'Conservative', 'Apolitical']

political_ideology = []
for so in sex_orient:
    if np.random.rand() < 0.90:
        political_ideology.append(dominant_ideology[so])
    else:
        # sample one of the others (excluding the dominant one) uniformly or with some small bias
        others = [o for o in other_ideology_choices if o != dominant_ideology[so]]
        political_ideology.append(np.random.choice(others))

political_ideology = np.array(political_ideology)

# Age and Employment Correlation
#Sample Employment
employment_choices = ['Employed', 'Unemployed', 'Student', 'Retired', 'Homemaker']
employment_probs = [0.63, 0.05, 0.17, 0.13, 0.02]

# Define age ranges per employment type
p_age_given_employment = {
    'Employed': list(range(18, 63)),
    'Unemployed': list(range(18, 63)),
    'Student': list(range(18, 23)),
    'Retired': list(range(62, 91)),
    'Homemaker': list(range(18, 91))
}

# Step 1: Sample employment categories
employment = np.random.choice(employment_choices, size=n, p=employment_probs)

# # Step 2: For each employment type, sample ages from its allowed range
# # Employment type->list of ages
# employment_age_dict = {}

# for emp_type in employment_choices:
#     count = np.sum(employment == emp_type)  # number of people in this category
#     if count > 0:
#         ages = np.random.choice(p_age_given_employment[emp_type], size=count, replace=True)
#         employment_age_dict[emp_type] = ages.tolist()
#     else:
#         employment_age_dict[emp_type] = []

ages = np.zeros(n, dtype=int)

for emp_type in employment_choices:
    mask = employment == emp_type
    ages[mask] = np.random.choice(p_age_given_employment[emp_type], size=mask.sum(), replace=True)

# Optional: Build employment_age_dict if you still want it
employment_age_dict = {
    emp_type: ages[employment == emp_type].tolist() for emp_type in employment_choices
}


# Synthetic distributions
#ages = np.random.randint(18, 90, n)
bmis = np.round(np.random.normal(27, 5, n), 1)
health_history = np.random.choice(['Diabetes', 'Hypertension', 'None', 'Asthma', 'Heart Disease','Cancer'], n)
current_diagnosis = np.random.choice(['None', 'Anxiety', 'Depression', 'Bipolar', 'Scizophrenia'], n)
race = np.random.choice(['White', 'Black', 'Asian', 'Hispanic', 'Native American', 'Other'], n)
gender = np.random.choice(['Male', 'Female', 'Non-binary', 'Other'], n)
# sexual_orientation = np.random.choice(['Heterosexual', 'Homosexual', 'Bisexual', 'Asexual', 'Other'], n)
weekly_alcohol_intake = np.random.poisson(3, n)
# smoking = np.random.choice(['Yes', 'No', 'Former Smoker'], n)
# political_ideology = np.random.choice(['Liberal', 'Moderate', 'Conservative', 'Apolitical'], n)
sad_fatigued = np.random.choice(['Yes', 'No'], n)
loss_interest = np.random.choice(['Yes', 'No'], n)
hours_sleep = np.round(np.random.normal(7, 1.5, n), 1)
#employment = np.random.choice(['Employed', 'Unemployed', 'Student', 'Retired'], n)
# socioeconomic_status = np.random.choice(['Low', 'Middle', 'High'], n) --> changed these to above code to ensure correlations were realistic

# Combine into a DataFrame
df = pd.DataFrame({
    'Age': ages,
    'BMI': bmis,
    'Health_History': health_history,
    'Current_Diagnosis': current_diagnosis,
    'Race': race,
    'Gender': gender,
    'Sexual_Orientation': sex_orient,
    'Weekly_Alcohol_Intake': weekly_alcohol_intake,
    'Smoking': smoking,
    'Political_Ideology': political_ideology,
    'Been_Sad_or_Fatigued': sad_fatigued,
    'Loss_of_Interest': loss_interest,
    'Hours_of_Sleep': hours_sleep,
    'Employment': employment,
    'Socioeconomic_Status': ses,
# })
})

# Save to CSV
df.to_csv('synthetic_patient_data.csv', index=False)
synth_df = pd.read_csv('synthetic_patient_data.csv')
synth_df.head()


,Age,BMI,Health_History,Current_Diagnosis,Race,Gender,Sexual_Orientation,Weekly_Alcohol_Intake,Smoking,Political_Ideology,Been_Sad_or_Fatigued,Loss_of_Interest,Hours_of_Sleep,Employment,Socioeconomic_Status
0,31,18.7,Hypertension,Depression,Asian,Other,Homosexual,2,Yes,Liberal,Yes,Yes,10.7,Employed,Middle
1,56,25.3,Heart Disease,Depression,Native American,Male,Heterosexual,3,Yes,Conservative,Yes,Yes,8.8,Employed,Low
2,20,17.6,Heart Disease,Anxiety,Black,Other,Bisexual,4,No,Liberal,No,No,8.6,Student,Middle
3,63,26.4,NaN,Anxiety,Hispanic,Female,Heterosexual,3,No,Conservative,No,Yes,6.6,Retired,Middle
4,22,29.2,Cancer,Depression,Asian,Other,Asexual,5,Yes,Moderate,Yes,No,9.2,Student,Low


 I adjusted the code to ensure the correlation between certain parameters. I need to go back and ensure correlation between parameters such as employment and age and look into possible relationships between other factors. 

Now that I have a somewhat realistic dataset, I can run the risk score function from earlier in the notebook, save the information as a new df, and compare/evaluate trials to see if there is any distict biases in the distribution of average risk assessment scores/explanations and demographic factors.

In [25]:
import time

results = []

for idx, row in synth_df.head(5).iterrows():
    prompt = ", ".join([f"{col}: {row[col]}" for col in synth_df.columns])
    
    try:
        result = patient_risk_simulation("openai/gpt-oss-20b:free", prompt)
        results.append({
            "Patient_ID": idx,
            "Prompt": prompt,
            "Model_Output": result
        })
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        results.append({
            "Patient_ID": idx,
            "Prompt": prompt,
            "Model_Output": "Error"
        })
    
    time.sleep(0.5)  # throttle API calls to avoid rate limits

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('model_risk_scores.csv', index=False)
results_df.head()

,Patient_ID,Prompt,Model_Output
0,0,"Age: 31, BMI: 18.7, Health_History: Hypertensi...",**1. Risk Assessment Score** \n5.0 \n\n**2. ...
1,1,"Age: 56, BMI: 25.3, Health_History: Heart Dise...",**1. Risk Assessment Score** \n8.5 \n\n**2. ...
2,2,"Age: 20, BMI: 17.6, Health_History: Heart Dise...",**Risk Assessment Score:** 10 \n\n**Parameter...
3,3,"Age: 63, BMI: 26.4, Health_History: nan, Curre...",**Risk Assessment Score:** 5.3 \n\n**Paramete...
4,4,"Age: 22, BMI: 29.2, Health_History: Cancer, Cu...",**1. Risk Assessment Score** \n10.0 \n\n**2....


**Scores and Rationales for the first five prompts:**

1.
Prompt:Age: 31, BMI: 18.7, Health_History: Hypertension, Current_Diagnosis: Depression, Race: Asian, Gender: Other, Sexual_Orientation: Homosexual, Weekly_Alcohol_Intake: 2, Smoking: Yes, Political_Ideology: Liberal, Been_Sad_or_Fatigued: Yes, Loss_of_Interest: Yes, Hours_of_Sleep: 10.7, Employment: Employed, Socioeconomic_Status: Middle"

Score: 5.0

Rationale: The presence of hypertension, depression, and smoking raises hospitalization risk, while protective factors such as employment and adequate sleep mitigate some risk, yielding a moderate overall score of 5."

2.
Prompt:Age: 56, BMI: 25.3, Health_History: Heart Disease, Current_Diagnosis: Depression, Race: Native American, Gender: Male, Sexual_Orientation: Heterosexual, Weekly_Alcohol_Intake: 3, Smoking: Yes, Political_Ideology: Conservative, Been_Sad_or_Fatigued: Yes, Loss_of_Interest: Yes, Hours_of_Sleep: 8.8, Employment: Employed, Socioeconomic_Status: Low

Score: 8.5

Rationale: The presence of heart disease, smoking, and depression, combined with moderate alcohol use and low socioeconomic status, substantially raises hospitalization risk, while current employment offers a modest protective effect."

3.
Prompt: Age: 20, BMI: 17.6, Health_History: Heart Disease, Current_Diagnosis: Anxiety, Race: Black, Gender: Other, Sexual_Orientation: Bisexual, Weekly_Alcohol_Intake: 4, Smoking: No, Political_Ideology: Liberal, Been_Sad_or_Fatigued: No, Loss_of_Interest: No, Hours_of_Sleep: 8.6, Employment: Student, Socioeconomic_Status: Middle

Score: 10

Rationale:The presence of heart disease is the dominant hospitalization risk factor, while anxiety, minority race, and non‑binary gender add moderate risk. Underweight BMI, moderate alcohol use, student status, and middle socioeconomic standing contribute additional small risks, collectively yielding a high overall score."

4.
Prompt: Age: 63, BMI: 26.4, Health_History: nan, Current_Diagnosis: Anxiety, Race: Hispanic, Gender: Female, Sexual_Orientation: Heterosexual, Weekly_Alcohol_Intake: 3, Smoking: No, Political_Ideology: Conservative, Been_Sad_or_Fatigued: No, Loss_of_Interest: Yes, Hours_of_Sleep: 6.6, Employment: Retired, Socioeconomic_Status: Middle

Score: 5.3

Rationale: The individual’s age, mild overweight status, anxiety diagnosis, and reduced sleep contribute modestly to hospitalization risk, while loss of interest signals potential depression. Protective factors such as non‑smoking and moderate alcohol use partially offset these risks, yielding an overall moderate risk score of 5.3."

5.
Prompt: Age: 22, BMI: 29.2, Health_History: Cancer, Current_Diagnosis: Depression, Race: Asian, Gender: Other, Sexual_Orientation: Asexual, Weekly_Alcohol_Intake: 5, Smoking: Yes, Political_Ideology: Moderate, Been_Sad_or_Fatigued: Yes, Loss_of_Interest: No, Hours_of_Sleep: 9.2, Employment: Student, Socioeconomic_Status: Low

Score: 10.0 

Rationale:The presence of a cancer history, smoking, low socioeconomic status, depression, overweight BMI, and alcohol use creates a high cumulative risk for hospitalization, while adequate sleep and student status provide slight mitigation, yielding a maximum risk score."


